In [ ]:
def _MA_screen(symbol, df, x):
    df['SMA'] = round(df['Adj Close'].rolling(window=x).mean(), 2)  # Simple Moving Average
    # TEMA triple exponential moving average | EMA Exponential Moving Average
    df['EMA1'] = round(df['Adj Close'].ewm(span=x, adjust=False).mean(), 2)
    df['EMA2'] = round(df['EMA1'].ewm(span=x, adjust=False).mean(), 2)
    df['EMA3'] = round(df['EMA2'].ewm(span=x, adjust=False).mean(), 2)
    df['TEMA'] = 3 * (df['EMA1'] - df['EMA2']) + df['EMA3']

    # up/down from previous 2 days
    df['SMA_2'] = df.SMA.shift(2)
    df['TEMA_2'] = df.TEMA.shift(2)
    df['SMA_up'] = df.apply(lambda x: '+' if x['SMA'] >= x['SMA_2'] else '-', axis=1)
    df['TEMA_up'] = df.apply(lambda x: '+' if x['TEMA'] >= x['TEMA_2'] else '-', axis=1)

    # reverse up/down
    df['SMA_up_1'] = df.SMA_up.shift(1)
    df['TEMA_up_1'] = df.TEMA_up.shift(1)
    df = df.dropna(subset=['SMA', 'SMA_2', 'TEMA', 'TEMA_2', 'SMA_up_1', 'TEMA_up_1'])
    df['SMA_up_rev'] = df.apply(lambda x: x['SMA_up'] if x['SMA_up'] != x['SMA_up_1'] else '', axis=1)
    df['TEMA_up_rev'] = df.apply(lambda x: x['TEMA_up'] if x['TEMA_up'] != x['TEMA_up_1'] else '', axis=1)

    SMA_seq = ''
    TEMA_seq = ''
    seq = 0
    for i in df.index[::-1]:  # look backwards
        seq += 1
        if bool(SMA_seq) & bool(TEMA_seq):
            break
        if (not SMA_seq) & bool(df['SMA_up_rev'][i]):
            SMA_seq = df['SMA_up_rev'][i] + str(seq)
        if (not TEMA_seq) & bool(df['TEMA_up_rev'][i]):
            TEMA_seq = df['TEMA_up_rev'][i] + str(seq)
    return [symbol, TEMA_seq, SMA_seq]


In [ ]:
def _MA_screen_list(stockList, x, filename):
    stockList = list(set(stockList))  # make elements unique
    stockList = sorted(stockList)
    rows = []
    for symbol in stockList:
        try:
            with contextlib.redirect_stdout(io.StringIO()):
                df_yh = yf.download(symbol, period='120d')
            row = _MA_screen(symbol, df_yh, x)
            rows.append(row)
        except Exception:
            print(symbol)
    out_df = pd.DataFrame(rows, columns=['symbol', 'TEMA_seq', 'SMA_seq'])

    out_df.TEMA_seq = out_df.TEMA_seq.astype(float)
    out_df.SMA_seq = out_df.SMA_seq.astype(float)
    out_df = out_df.sort_values(by=['TEMA_seq', 'SMA_seq'], ascending=False)

    print(out_df)
    out_df.to_csv(DIR + filename, index=False)

In [ ]:
def _MA_screen_list_h(stockList, x, filename):
    stockList = list(set(stockList))  # make elements unique
    stockList = sorted(stockList)
    rows = []
    for symbol in stockList:
        try:
            with contextlib.redirect_stdout(io.StringIO()):
                df_yh = yf.download(symbol, period="20d", interval="1h")
            row = _MA_screen(symbol, df_yh, x)
            rows.append(row)
        except Exception:
            print(symbol)
    out_df = pd.DataFrame(rows, columns=['symbol', 'TEMA_seq', 'SMA_seq'])

    out_df.TEMA_seq = out_df.TEMA_seq.astype(float)
    out_df.SMA_seq = out_df.SMA_seq.astype(float)
    out_df = out_df.sort_values(by=['TEMA_seq', 'SMA_seq'], ascending=False)

    print(out_df)
    out_df.to_csv(DIR + filename, index=False)

In [ ]:
# C reverse up +1: buy to close C
import contextlib
import io

import pandas as pd
import yfinance as yf

yf.pdr_override()
pd.options.mode.chained_assignment = None  # default='warn'

x = 20
DIR = 'E:\Google Drive\data'

df = pd.read_csv(DIR + '\out_fidelity.csv')
df = df[(df['Option'] == 'C')]
stockList = df['Ticker'].tolist()

_MA_screen_list_h(stockList, x, '\out_Python_MA_C.csv')


In [ ]:
# P C2 reverse dn -1: buy to close P or sell C
df = pd.read_csv('E:\Google Drive\data\out_fidelity.csv')
df = df[(df['Option'] == 'P') | df['C2'].notnull()]
stockList = df['Ticker'].tolist()

_MA_screen_list_h(stockList, x, '\out_Python_MA_P.csv')

In [ ]:
# Scan index companies
import contextlib
import io

import pandas as pd
import yfinance as yf

yf.pdr_override()
pd.options.mode.chained_assignment = None  # default='warn'

x = 20
DIR = 'E:\Google Drive\data'

from pandas import DataFrame
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
lst = stock_data.get_stocks_by_index('DOW JONES')  # NASDAQ 100 | S&P 500 | DOW JONES
df = DataFrame(lst)
stockList = df['symbol'].tolist()  # get list from a column

_MA_screen_list(stockList, x, '\out_Python_MA_SP.csv')


In [ ]:
import datetime as dt

def _pickUpDn(limitDay, filename):
    out_df = pd.read_csv(DIR + filename)
    today = dt.datetime.now().strftime('%d')
    upList = out_df[(0 <= out_df['TEMA_seq']) & (out_df['TEMA_seq'] <= limitDay)]  # & (out_df['SMA_seq'] < -9)
    print(upList)
    upList['symbol'].to_csv('..\data\out_PythonRevUp' + today + '.txt', index=False, header=False)

    dnList = out_df[(-limitDay <= out_df['TEMA_seq']) & (out_df['TEMA_seq'] <= 0)]  # & (out_df['SMA_seq'] > 9)
    print(dnList)
    dnList['symbol'].to_csv('..\data\out_PythonRevDn' + today + '.txt', index=False, header=False)


In [ ]:
_pickUpDn(3, '\out_Python_MA_SP.csv')

In [ ]:
# Scan yuanjing list
df = pd.read_csv('..\data\in_yuanjing_list.csv', sep='[";"]', engine='python')
stockList = []
for i in df.index:
    symbolList = df['Symbol'][i]
    for x in symbolList.split(','):
        stockList.append(x.split(':')[-1])
        
filename = '\out_Python_MA.csv'
_MA_screen_list(stockList, x, filename)
_pickUpDn(limitDay, filename)


In [ ]:
# 1 symbol
import contextlib
import io

import pandas as pd
import yfinance as yf

yf.pdr_override()
pd.options.mode.chained_assignment = None  # default='warn'

symbol = 'NVDA'
with contextlib.redirect_stdout(io.StringIO()):
#     df_yh = yf.download(symbol, period="120d")
    df_yh = yf.download(symbol, period="20d", interval="1h")
print(_MA_screen(symbol, df_yh, 20))
